## Dependencies

In [1]:
# Install Modules
!pip install keras_tuner -q

# Load Dependencies
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import tensorflow as tf
import keras_tuner as kt
import warnings
import numpy as np
from collections import Counter
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')

# Disable all warnings
warnings.filterwarnings("ignore")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 4.5 MB/s eta 0:00:00


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


## Load Data

In [2]:
def load_join(select, loaded):
    selected_df = pd.read_csv("https://www.dropbox.com/s/" + select + "?dl=1", index_col="review_index")
    if loaded is not None:
        selected_df = pd.merge(loaded, selected_df, on='review_index')
    return selected_df

In [3]:
load_metadata = True
load_ratings = True
load_opinions = True
load_descriptions = True
load_world_happiness_report = False

index_df = None
if load_metadata == True:
    load_index = "goqyfyeu2qvpsmo/metadata_df.csv"
    index_df = load_join(load_index, index_df)
if load_ratings == True:
    load_index = "9j2j86xwqrmrljx/ratings_df.csv"
    index_df = load_join(load_index, index_df)
if load_opinions == True:
    load_index = "q8v9f6rbb4z12df/opinion_df.csv"
    index_df = load_join(load_index, index_df)
if load_descriptions == True:
    load_index = "hnp8sebleh6dzgt/descriptions_df.csv"
    index_df = load_join(load_index, index_df)
    
if load_world_happiness_report == True:
    WHR_df = pd.read_csv("https://www.dropbox.com/s/jyr4e7fleevrb1s/WHR2023.csv?dl=1")

In [62]:
user_input_df = index_df.drop(['firm', 'date_review', 'job_title', 'location', 'current',	'work_life_balance',	'culture_values',	'diversity_inclusion',
                               'career_opp',	'comp_benefits',	'senior_mgmt',	'recommend',	'ceo_approv',	'outlook'], axis=1).copy()

nn_working_df = index_df.drop(['date_review', 'job_title', 'location', 'headline', 'pros', 'cons'], axis=1).copy()

nn_op_working_df = index_df.drop(['date_review', 'job_title', 'location'], axis=1).copy()

## Cleaning Data Attempt - Half Hot-Ended Remove NaN

In [5]:
working_half_hotend_df = nn_working_df.copy()
columns_to_convert = ['overall_rating', 'work_life_balance', 'culture_values',
                      'diversity_inclusion', 'career_opp', 'comp_benefits',
                      'senior_mgmt']

working_half_hotend_df[columns_to_convert] = working_half_hotend_df[columns_to_convert].astype(float)

In [6]:
cutoff_value = 3000
firms_to_replace = working_half_hotend_df['firm'].value_counts()[working_half_hotend_df['firm'].value_counts() < cutoff_value].index.tolist()

# Replace in dataframe
for firm in firms_to_replace:
    working_half_hotend_df['firm'] = working_half_hotend_df['firm'].replace(firm,"Other")
    
# Check to make sure binning was successful
print(working_half_hotend_df['firm'].value_counts())
print("These ar how many samples that got 'firm' changed to Other: ", len(working_half_hotend_df[working_half_hotend_df['firm'] == "Other"]))
print("Number of firms removed: ", len(firms_to_replace))
print("Remaining number of firms: ", len(working_half_hotend_df['firm'].value_counts()))

Other                      178782
IBM                         60436
McDonald-s                  49450
Deloitte                    46995
EY                          34050
PwC                         33227
Oracle                      31941
Microsoft                   26675
J-P-Morgan                  25814
KPMG                        24815
Apple                       20797
Citi                        18726
Google                      15995
SAP                         14344
HSBC-Holdings               13893
Tesco                       12149
Marriott-International      10409
Barclays                     9710
Thomson-Reuters              9553
American-Express             9349
Morgan-Stanley               9093
Goldman-Sachs                8808
Vodafone                     8321
Salesforce                   8234
Pizza-Hut                    7592
BNY-Mellon                   6630
Deutsche-Bank                6388
Hilton                       6155
J-Sainsbury                  5925
GlaxoSmithKlin

In [7]:
# Cutoff value to make a list of values in the 'current' column to put into "other" catagory
cutoff_value = 10
current_to_replace = working_half_hotend_df['current'].value_counts()[working_half_hotend_df['current'].value_counts() < cutoff_value].index.tolist()

# Replace in dataframe
for review in current_to_replace:
    working_half_hotend_df['current'] = working_half_hotend_df['current'].replace(review,"Other")

# Check to make sure binning was successful
working_half_hotend_df['current'].value_counts()

Current Employee                        209599
Former Employee                         146133
Current Employee, more than 1 year       82749
Current Employee, more than 3 years      66471
Former Employee, more than 1 year        65687
Current Employee, less than 1 year       49603
Former Employee, more than 3 years       43614
Former Employee, less than 1 year        41874
Current Employee, more than 5 years      40155
Current Employee, more than 10 years     25029
Former Employee, more than 5 years       23017
Current Employee, more than 8 years      18506
Former Employee, more than 10 years      15411
Former Employee, more than 8 years       10686
Other                                       32
Name: current, dtype: int64

In [8]:
print("Columns before processing: ", len(working_half_hotend_df))
working_half_hotend_df.dropna(subset=columns_to_convert, inplace=True)
print("Column after dropping NaN values: ", len(working_half_hotend_df))
working_half_hotend_df.nunique()

Columns before processing:  838566
Column after dropping NaN values:  133863


firm                   59
current                15
overall_rating          5
work_life_balance       5
culture_values          5
diversity_inclusion     5
career_opp              5
comp_benefits           5
senior_mgmt             5
recommend               3
ceo_approv              4
outlook                 4
dtype: int64

In [9]:
X = pd.get_dummies(working_half_hotend_df.drop('overall_rating', axis = 1)).drop(['recommend_o', 'ceo_approv_o', 'outlook_o'], axis = 1)
y = pd.get_dummies(working_half_hotend_df['overall_rating'].astype(int))

X.columns

Index(['work_life_balance', 'culture_values', 'diversity_inclusion',
       'career_opp', 'comp_benefits', 'senior_mgmt', 'firm_ASDA',
       'firm_American-Express', 'firm_Aon', 'firm_Apple', 'firm_AstraZeneca',
       'firm_BNP-Paribas', 'firm_BNY-Mellon', 'firm_BT',
       'firm_Bain-and-Company', 'firm_Barclays', 'firm_Bloomberg-L-P',
       'firm_Booking-com', 'firm_Boots', 'firm_Boston-Consulting-Group',
       'firm_CBRE', 'firm_Capita', 'firm_Citi', 'firm_Deloitte',
       'firm_Deutsche-Bank', 'firm_EY', 'firm_GlaxoSmithKline',
       'firm_Goldman-Sachs', 'firm_Google', 'firm_Grant-Thornton',
       'firm_HSBC-Holdings', 'firm_Hays', 'firm_Hilton', 'firm_Hyatt',
       'firm_IBM', 'firm_J-P-Morgan', 'firm_J-Sainsbury', 'firm_KPMG',
       'firm_LinkedIn', 'firm_Lloyds-Banking-Group', 'firm_Marks-and-Spencer',
       'firm_Marriott-International', 'firm_McDonald-s',
       'firm_McKinsey-and-Company', 'firm_Mercer', 'firm_Microsoft',
       'firm_Morgan-Stanley', 'firm_NHS', '

In [10]:
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

X_train_scaled.shape

(100397, 88)

## NN Attempt - Compile, Train, & Evaluate Default Model

In [11]:
# Define the model
nn_default = tf.keras.models.Sequential()

# Add the masking layer
#nn_default.add(tf.keras.layers.Masking(mask_value=10, input_shape=(X_train_scaled.shape[1],)))

nn_default.add(tf.keras.layers.Dense(units=100, activation='relu', input_dim=X_train_scaled.shape[1]))

nn_default.add(tf.keras.layers.Dense(units=80, activation='relu'))

# Add the output layer
nn_default.add(tf.keras.layers.Dense(units=5, activation='sigmoid'))

# Check the structure of the model
nn_default.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               8900      
                                                                 
 dense_1 (Dense)             (None, 80)                8080      
                                                                 
 dense_2 (Dense)             (None, 5)                 405       
                                                                 
Total params: 17,385
Trainable params: 17,385
Non-trainable params: 0
_________________________________________________________________


In [12]:
# Compile the model
nn_default.compile(loss='mse', optimizer='adam', metrics=['accuracy'])

# Train the model
history = nn_default.fit(X_train_scaled, y_train, epochs=20, verbose=1, batch_size=64)

Epoch 1/20
1569/1569 [==============================] - 6s 3ms/step - loss: 0.0962 - accuracy: 0.6383
Epoch 2/20
1569/1569 [==============================] - 3s 2ms/step - loss: 0.0900 - accuracy: 0.6634
Epoch 3/20
1569/1569 [==============================] - 4s 2ms/step - loss: 0.0891 - accuracy: 0.6682
Epoch 4/20
1569/1569 [==============================] - 3s 2ms/step - loss: 0.0885 - accuracy: 0.6705
Epoch 5/20
1569/1569 [==============================] - 4s 3ms/step - loss: 0.0879 - accuracy: 0.6733
Epoch 6/20
1569/1569 [==============================] - 4s 3ms/step - loss: 0.0874 - accuracy: 0.6763
Epoch 7/20
1569/1569 [==============================] - 4s 2ms/step - loss: 0.0868 - accuracy: 0.6777
Epoch 8/20
1569/1569 [==============================] - 2s 2ms/step - loss: 0.0864 - accuracy: 0.6803
Epoch 9/20
1569/1569 [==============================] - 2s 1ms/step - loss: 0.0860 - accuracy: 0.6828
Epoch 10/20
1569/1569 [==============================] - 2s 1ms/step - loss: 0.085

In [13]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_default.evaluate(X_test_scaled,y_test,verbose=1, batch_size=64)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

523/523 [==============================] - 1s 967us/step - loss: 0.0918 - accuracy: 0.6588
Loss: 0.09178947657346725, Accuracy: 0.6587880253791809


In [14]:
predictions = pd.DataFrame(nn_default.predict(X_test_scaled))

1046/1046 [==============================] - 1s 832us/step


In [15]:
predictions = predictions.rename(columns={0: 1, 1: 2, 2: 3, 3: 4, 4: 5})
predictions['y_predict'] = predictions.idxmax(axis=1)
predictions['y_actual'] = (y_test.idxmax(axis=1)).reset_index(drop=True)
predictions[1:10]

,1,2,3,4,5,y_predict,y_actual
1,0.000035,3.733816e-05,0.038713,0.694281,0.232375,4,4
2,0.000581,2.193054e-05,0.120306,0.547364,0.360868,4,5
3,0.006839,1.425769e-02,0.238363,0.577762,0.060083,4,1
4,0.717077,2.137113e-01,0.026523,0.007406,0.006450,1,1
5,0.000041,1.825487e-06,0.022788,0.486622,0.452654,4,4
6,0.013532,5.443876e-02,0.684440,0.211661,0.018587,3,3
7,0.829272,6.171545e-02,0.038280,0.013916,0.018151,1,1
8,0.000009,2.846333e-08,0.018283,0.626462,0.283581,4,5
9,0.000115,1.105093e-04,0.064042,0.762625,0.143783,4,4


In [16]:
difference = 0

for index, row in predictions.iterrows():
    # Get the absolute difference between 'y_predict' and 'y_actual' columns
    diff = abs(row['y_predict'] - row['y_actual'])
    
    # Add the difference to the total
    difference += diff

difference = difference / len(predictions)
print("Mean points delta actual value: ", difference)

Mean points delta actual value:  0.375186756708301


Hot Ended - Mean points delta actual value:  0.43129239369973577

Half Hot Ended Remove NaN - Mean points delta actual value:  0.36371242455028985

Half Hot Ended Mask NaN - Mean points delta actual value:  0.4334150599593593

## Scaping Key Words from User Inputs

In [17]:
working_user_input_df = user_input_df.copy()
working_user_input_df = working_user_input_df[working_user_input_df.index.isin(working_half_hotend_df.index)]
print("Rows in dataset prior ro removing rows with NaN values: ", len(user_input_df))
print("Rows in dataset after removing rows with NaN values: ", len(working_user_input_df))
working_user_input_df.head(1)

Rows in dataset prior ro removing rows with NaN values:  838566
Rows in dataset after removing rows with NaN values:  133863


,overall_rating,headline,pros,cons
review_index,,,,
50,2,The people both make and destroy this place,"Great people in some places, excellent Christm...","Poor pay, huge gap for pay between senior mana..."


In [18]:
def text_tokenizer(text):
    # Tokenize the text into individual words
    words = word_tokenize(text.lower())

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    filtered_words = [word for word in words if word.isalnum() and word not in stop_words]

    return filtered_words

# Text processor takes dataframe of only 1 column of strings, tokenizes it, and returns series of comma separated words from each string.

In [19]:
def word_counter(series_column, n):
  
  all_words = [word for sublist in series_column for word in sublist]
  word_counts = Counter(all_words)

  most_common_words = word_counts.most_common(n)

  return most_common_words

# Takes in tokenized series and returns the top "value of 'n'" words as a series.

In [20]:
def column_processor(rating, attribute, n):
  index_user_input_df = working_user_input_df[working_user_input_df['overall_rating'] == rating].copy().drop(['overall_rating'], axis = 1)
  isolated = index_user_input_df[attribute].rename(str(attribute)+ "_" + str(rating))
  isolated = isolated.reset_index(drop=True)
  isolated = isolated.dropna().astype(str)
  preprocessed_data = isolated.apply(text_tokenizer)
  word_count_list = word_counter(preprocessed_data, n)

  return word_count_list

# Takes requested rating and attribute, finds the column, and calls to the function that tokenizes text, calls to the function that counts words. Returns series.

In [133]:
parsed_df = pd.DataFrame()

return_top_counter = 750

for i in range(1, 6):
    for attribute in working_user_input_df.drop(['overall_rating'], axis=1):
        index_series_to_df = pd.DataFrame(column_processor(i, attribute, return_top_counter), columns=[str(attribute) + '_' + str(i) + '_words', str(attribute) + '_' + str(i) + '_count'])
        index_series_to_df = index_series_to_df.rename_axis('word_index')

        if parsed_df.empty:
            parsed_df = index_series_to_df
        else:
            parsed_df = pd.merge(parsed_df, index_series_to_df, on='word_index')

In [134]:
parsed_words_df = parsed_df.drop(columns=[col for col in parsed_df.columns if 'count' in col])
duplicate_words_list = parsed_words_df.values.flatten().tolist()
duplicate_words_list = [word for word in duplicate_words_list if duplicate_words_list.count(word) > 1]
duplicate_words_list = list(set(duplicate_words_list))
print(len(duplicate_words_list))

1475


In [153]:
parsed_words_optimized_df = parsed_words_df.copy()
for column in parsed_words_optimized_df.columns:
    parsed_words_optimized_df[column] = parsed_words_optimized_df[column].apply(lambda x: '' if x in duplicate_words_list else x)

parsed_words_optimized_df = parsed_words_optimized_df.apply(lambda x: x.sort_values().values)
parsed_words_optimized_df = parsed_words_optimized_df.apply(lambda x: sorted(x, key=bool, reverse=True))
parsed_words_optimized_df = parsed_words_optimized_df[~(parsed_words_optimized_df == '').all(axis=1)]
parsed_words_optimized_df.head(10)

,headline_1_words,pros_1_words,cons_1_words,headline_2_words,pros_2_words,cons_2_words,headline_3_words,pros_3_words,cons_3_words,headline_4_words,pros_4_words,cons_4_words,headline_5_words,pros_5_words,cons_5_words
word_index,,,,,,,,,,,,,,,
0,abused,advantage,7,beginning,art,alone,adequate,availability,accountability,aon,active,adapt,2020,accommodating,applicable
1,administrative,air,abuse,british,assistance,asking,aight,camaraderie,external,ba,benifits,assigned,accounts,continue,cold
2,appalling,attractive,act,careful,cafe,cuts,builder,casual,inefficient,becoming,businesses,band,ae,cooperative,commute
3,atrocious,bearable,allow,carefully,deals,deliver,dependant,coverage,rise,commercial,competent,comparison,ai,developing,complain
4,azure,book,answer,carrier,direct,drama,downs,prestigious,stand,deutsche,cross,confusing,ambassador,encourage,complexity
5,backwards,buildings,behavior,cog,finish,heard,ehh,rates,,foundation,learnings,demand,apprentice,encourages,downside
6,bone,buy,bloomberg,debenhams,guaranteed,promised,losing,soft,,grind,organised,demands,army,ensure,downsides
7,brutal,country,board,draining,immediate,regardless,neutral,,,improvements,sectors,depend,awsome,equality,facing
8,burn,drink,bullied,execution,main,related,reporting,,,kickstart,transport,merit,bain,joined,fault


## Processing User Input strings into columns with word counts

In [178]:
nn_op_working_df[columns_to_convert] = nn_op_working_df[columns_to_convert].astype(float)

cutoff_value = 3000
firms_to_replace = nn_op_working_df['firm'].value_counts()[nn_op_working_df['firm'].value_counts() < cutoff_value].index.tolist()

for firm in firms_to_replace:
    nn_op_working_df['firm'] = nn_op_working_df['firm'].replace(firm,"Other")

cutoff_value2 = 10
current_to_replace = nn_op_working_df['current'].value_counts()[nn_op_working_df['current'].value_counts() < cutoff_value2].index.tolist()

for review in current_to_replace:
    nn_op_working_df['current'] = nn_op_working_df['current'].replace(review,"Other")

nn_op_working_df.dropna(subset=columns_to_convert, inplace=True)
nn_op_working_df = nn_op_working_df.reset_index(drop=True)
print(len(nn_op_working_df))

133863


In [139]:
parsed_words_df.shape

(750, 15)

In [193]:
word_counter_df = pd.DataFrame() # Full DF to be merged

for index, row in nn_op_working_df.iterrows():
    index_df = pd.DataFrame() # Individual row
    if index % 1000 == 0:
        percent_total = (index)/len(nn_op_working_df)*100

        print(f"\rProcessing {index} of {len(nn_op_working_df)} COMPLETE {percent_total:.2f}%", end="")

        print(f"\br********************")
    for column in ['headline', 'pros', 'cons']:
        attribute_index_df = pd.DataFrame() # column in Individual row
        working_index = text_tokenizer(str(row[column]))
        for i in range(1, 6):
            attribute_rating_counter = 0 # Value to be acdded
            checking_index = parsed_words_optimized_df[parsed_words_optimized_df[str(column) + '_' + str(i) + '_words'] != ''][str(column) + '_' + str(i) + '_words'].tolist()
            for word in working_index:
                if word in checking_index: # count instances of matching words
                    attribute_rating_counter = attribute_rating_counter + 1
            #print("index #", index, "___comparing ", len(working_index), " words against ", len(checking_df), " in ", str(column) + '_' + str(i) + '_words', attribute_rating_counter, " matches")
            attribute_index_df[str(column) + '_' + str(i) + '_counter'] = [attribute_rating_counter] # creates a new column for each attribute and rating counter
        if index_df.empty:
            index_df = attribute_index_df
        else:
            index_df = pd.merge(index_df, attribute_index_df, left_index=True, right_index=True)
    word_counter_df = pd.concat([word_counter_df, index_df], ignore_index=True)

word_counter_df

Processing 0 of 133863 COMPLETE 0.00%r********************


KeyboardInterrupt: ignored